In [12]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [4]:
from langchain.storage import InMemoryByteStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
loaders=[
    TextLoader("documents\huawei_installment.txt",encoding="utf-8"),
    TextLoader("documents\huawei_service.txt",encoding="utf-8"),
]
loaders

In [8]:
docs=[]
for loader in loaders:
    docs.extend(loader.load())
    
docs

[Document(metadata={'source': 'documents\\huawei_installment.txt'}, page_content='华为商城（VMALL）分期付款业务指南\n1. 业务概述\n\n华为分期是华为商城联合各大银行及金融机构（如花呗、招商银行、平安银行等）推出的“先消费、后付款”的金融服务。用户在购买华为产品（如 Mate 60 系列、Pura 系列等）时，可将订单金额分为若干期缴纳。\n2. 支付方式与渠道\n\n目前华为商城支持以下主流分期工具：\n\n    花呗分期：支持 3 期、6 期、12 期。\n\n    银行信用卡分期：\n\n        合作银行：招商银行、中国工商银行、中国建设银行、中国农业银行等。\n\n        可选期数：通常提供 3、6、12、24 期选择。\n\n    掌上生活/其它银行 App 支付：通过跳转银行原生 App 完成分期操作。\n\n3. 费率与优惠说明\n3.1 费率标准\n\n    手续费计算：每期还款额 = 订单总金额 ÷ 分期期数 + 手续费。\n\n    费率浮动：具体手续费率以支付页面显示为准，不同银行费率略有差异。\n\n3.2 免息活动（重要）\n\n    0 首付 0 利息 0 手续费：华为经常针对新品推出“限时免息”活动。\n\n    适用条件：需满足指定的订单金额门槛（例如满 499 元可用）或指定机型。\n\n4. 办理流程\n\n    提交订单：在华为商城选择商品，进入结算页面。\n\n    选择分期：在支付方式中勾选“分期支付”。\n\n    确认期数：选择具体的期数（如 12 期免息），系统会实时显示每期应还金额。\n\n    身份验证：跳转至对应的支付平台（如支付宝或银行 H5 页面）进行生物识别或密码确认。\n\n    支付成功：订单状态更新为“待发货”。\n\n5. 常见问题 (FAQ)\n\n    Q：分期后可以提前还款吗？\n\n        A：可以。但部分银行可能会收取提前还款手续费，建议咨询发卡行。\n\n    Q：申请分期退货后，利息怎么算？\n\n        A：订单全额退款后，已收取的费用通常会按银行规则退回，未产生的期数自动取消。\n\n    Q：为什么我无法选择 24 期分期？\n

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(
  base_url="http://127.0.0.1:1234/v1",
  openai_api_key="lm-studio",
  model="text-embedding-bge-large-zh-v1.5",
  check_embedding_ctx_length=False
)

In [10]:
vectorstore=Chroma(
    collection_name="full_documents",embedding_function=embeddings
)

C:\Users\86151\AppData\Local\Temp\ipykernel_117336\1037862617.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore=Chroma(


In [13]:
#父文档的存储层
store=InMemoryByteStore()
id_key="doc_id"
#检索器（空启动）
retriever=MultiVectorRetriever(
    vectorstore=vectorstore,#向量存储库
    docstore=store, #文档存储库
    id_key=id_key,  #标识键，用于在 vectorstore 和 docstore 之间建立一一对应的映射关系
)

In [15]:
import uuid

doc_ids=[str(uuid.uuid4()) for _ in docs]
doc_ids

['9343e0f5-b143-4aca-ad2f-58cbf8a08171',
 '44e7247e-b9f0-47e9-9793-b7a196d0dd1c']

In [17]:
from langchain_text_splitters import CharacterTextSplitter

child_text_splitter=CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,  #计算块长度
    is_separator_regex=False, #是否为正则表达式
)

In [18]:
sub_docs=[]
for i,doc in enumerate(docs):
    _id=doc_ids[i]
    _sub_docs=child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key]=_id
    sub_docs.extend(_sub_docs)
sub_docs

Created a chunk of size 113, which is longer than the specified 100


[Document(metadata={'source': 'documents\\huawei_installment.txt', 'doc_id': '9343e0f5-b143-4aca-ad2f-58cbf8a08171'}, page_content='华为商城（VMALL）分期付款业务指南\n1. 业务概述'),
 Document(metadata={'source': 'documents\\huawei_installment.txt', 'doc_id': '9343e0f5-b143-4aca-ad2f-58cbf8a08171'}, page_content='华为分期是华为商城联合各大银行及金融机构（如花呗、招商银行、平安银行等）推出的“先消费、后付款”的金融服务。用户在购买华为产品（如 Mate 60 系列、Pura 系列等）时，可将订单金额分为若干期缴纳。\n2. 支付方式与渠道'),
 Document(metadata={'source': 'documents\\huawei_installment.txt', 'doc_id': '9343e0f5-b143-4aca-ad2f-58cbf8a08171'}, page_content='目前华为商城支持以下主流分期工具：\n\n    花呗分期：支持 3 期、6 期、12 期。\n\n    银行信用卡分期：\n\n        合作银行：招商银行、中国工商银行、中国建设银行、中国农业银行等。'),
 Document(metadata={'source': 'documents\\huawei_installment.txt', 'doc_id': '9343e0f5-b143-4aca-ad2f-58cbf8a08171'}, page_content='可选期数：通常提供 3、6、12、24 期选择。\n\n    掌上生活/其它银行 App 支付：通过跳转银行原生 App 完成分期操作。\n\n3. 费率与优惠说明\n3.1 费率标准'),
 Document(metadata={'source': 'documents\\huawei_installment.txt', 'doc_id': '9343e0f5-b143-4aca-ad2f-58cbf8a08171'

摘要总结

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

In [19]:
from langchain_openai import ChatOpenAI,OpenAI

model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.3
)

In [21]:
chain=(
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("请总结以下内容：{doc}")
    | model
    | StrOutputParser()
)

In [22]:
docs=[]
for loader in loaders:
    docs.extend(loader.load())
    
docs

[Document(metadata={'source': 'documents\\huawei_installment.txt'}, page_content='华为商城（VMALL）分期付款业务指南\n1. 业务概述\n\n华为分期是华为商城联合各大银行及金融机构（如花呗、招商银行、平安银行等）推出的“先消费、后付款”的金融服务。用户在购买华为产品（如 Mate 60 系列、Pura 系列等）时，可将订单金额分为若干期缴纳。\n2. 支付方式与渠道\n\n目前华为商城支持以下主流分期工具：\n\n    花呗分期：支持 3 期、6 期、12 期。\n\n    银行信用卡分期：\n\n        合作银行：招商银行、中国工商银行、中国建设银行、中国农业银行等。\n\n        可选期数：通常提供 3、6、12、24 期选择。\n\n    掌上生活/其它银行 App 支付：通过跳转银行原生 App 完成分期操作。\n\n3. 费率与优惠说明\n3.1 费率标准\n\n    手续费计算：每期还款额 = 订单总金额 ÷ 分期期数 + 手续费。\n\n    费率浮动：具体手续费率以支付页面显示为准，不同银行费率略有差异。\n\n3.2 免息活动（重要）\n\n    0 首付 0 利息 0 手续费：华为经常针对新品推出“限时免息”活动。\n\n    适用条件：需满足指定的订单金额门槛（例如满 499 元可用）或指定机型。\n\n4. 办理流程\n\n    提交订单：在华为商城选择商品，进入结算页面。\n\n    选择分期：在支付方式中勾选“分期支付”。\n\n    确认期数：选择具体的期数（如 12 期免息），系统会实时显示每期应还金额。\n\n    身份验证：跳转至对应的支付平台（如支付宝或银行 H5 页面）进行生物识别或密码确认。\n\n    支付成功：订单状态更新为“待发货”。\n\n5. 常见问题 (FAQ)\n\n    Q：分期后可以提前还款吗？\n\n        A：可以。但部分银行可能会收取提前还款手续费，建议咨询发卡行。\n\n    Q：申请分期退货后，利息怎么算？\n\n        A：订单全额退款后，已收取的费用通常会按银行规则退回，未产生的期数自动取消。\n\n    Q：为什么我无法选择 24 期分期？\n

In [23]:
summaries=chain.batch(docs,{"max_concurrency":5})
summaries

['华为商城（VMALL）的分期付款业务指南包括了业务概述、支付方式与渠道、费用与优惠说明、办理流程和常见问题等信息。用户可以通过华为商城购买华为产品时选择分期付款服务，享受灵活的还款方案。分期付款通常提供多种支付工具，如花呗、银行信用卡和掌上生活/其它银行 App 支付。华为定期推出“限时免息”活动，为用户提供优惠。办理分期付款需要提交订单，选择分期方式并进行身份验证，最后完成支付流程。常见问题包括分期后是否可以提前还款、申请分期退货后的利息计算以及无法选择 24 期分期的原因等。',
 '华为商城的退换货服务政策主要包括以下几点：\n\n1. **总则**：华为商城严格遵循国家《三包》法规，并提供“7天无理由退货、15天质量问题换货、一年保修”的售后保障服务。\n\n2. **退换货时效**：\n   - 7天无理由退货：自收货次日起7天内，若商品完好（未激活、无受损）的情况下可申请退货。\n   - 15天质量问题换货：自收货次日起15天内，若产品出现性能故障，经华为官方售后检测属实，可免费更换同型号同规格的新机。\n   - 一年保修：自收货次日起一年内，商品出现非人为损坏的性能故障，提供免费维修服务。\n\n3. **“商品完好”判定标准**：\n   - 申请7天无理由退货时，需满足以下条件：\n     - 产品未激活：手机、平板等涉及系统激活的产品，一旦联网激活，非质量问题不支持退货。\n     - 外观完好：产品机身、配件无划痕、无磨损、无拆解记录。\n     - 包装完整：外包装、说明书、保修卡、发票及赠品需一并退回。\n\n4. **特殊产品限制**：\n   - 个性化定制产品：如激光刻字版、特殊定制外壳产品，非质量问题不支持7天无理由退货。\n   - 卫生敏感类产品：如入耳式耳机，若包装封条已拆封且无质量问题，出于卫生考虑不支持退货。\n\n5. **退款流程**：\n   - 提交申请：在“我的订单”中点击“申请售后”，上传商品实拍照片。\n   - 寄回商品：审核通过后，用户需将商品寄回指定仓库（部分地区支持顺丰上门取件）。\n   - 仓库质检：收货后1-3个工作日内完成检测。\n   - 退款到账：质检合格后，款项将按原支付路径退回，预计3-5个工作日到账。']

In [25]:
vectorstore=Chroma(collection_name="summaries",embedding_function=embeddings)

store=InMemoryByteStore()
id_key="doc_id"

retriever=MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids=[str(uuid.uuid4()) for _ in docs]

In [26]:
summary_docs=[
    Document(page_content=s,metadata={id_key:doc_ids[i]})
    for i,s in enumerate(summaries)
]
summary_docs

[Document(metadata={'doc_id': 'cdabc3c7-380f-4480-ab07-55685934f544'}, page_content='华为商城（VMALL）的分期付款业务指南包括了业务概述、支付方式与渠道、费用与优惠说明、办理流程和常见问题等信息。用户可以通过华为商城购买华为产品时选择分期付款服务，享受灵活的还款方案。分期付款通常提供多种支付工具，如花呗、银行信用卡和掌上生活/其它银行 App 支付。华为定期推出“限时免息”活动，为用户提供优惠。办理分期付款需要提交订单，选择分期方式并进行身份验证，最后完成支付流程。常见问题包括分期后是否可以提前还款、申请分期退货后的利息计算以及无法选择 24 期分期的原因等。'),
 Document(metadata={'doc_id': '8a81550b-8430-4a84-afbc-8621ec247b51'}, page_content='华为商城的退换货服务政策主要包括以下几点：\n\n1. **总则**：华为商城严格遵循国家《三包》法规，并提供“7天无理由退货、15天质量问题换货、一年保修”的售后保障服务。\n\n2. **退换货时效**：\n   - 7天无理由退货：自收货次日起7天内，若商品完好（未激活、无受损）的情况下可申请退货。\n   - 15天质量问题换货：自收货次日起15天内，若产品出现性能故障，经华为官方售后检测属实，可免费更换同型号同规格的新机。\n   - 一年保修：自收货次日起一年内，商品出现非人为损坏的性能故障，提供免费维修服务。\n\n3. **“商品完好”判定标准**：\n   - 申请7天无理由退货时，需满足以下条件：\n     - 产品未激活：手机、平板等涉及系统激活的产品，一旦联网激活，非质量问题不支持退货。\n     - 外观完好：产品机身、配件无划痕、无磨损、无拆解记录。\n     - 包装完整：外包装、说明书、保修卡、发票及赠品需一并退回。\n\n4. **特殊产品限制**：\n   - 个性化定制产品：如激光刻字版、特殊定制外壳产品，非质量问题不支持7天无理由退货。\n   - 卫生敏感类产品：如入耳式耳机，若包装封条已拆封且无质量问题，出于卫生考虑不支持退货。\n\n5. **退款流程**：\n   - 提交申请：在“我的订单”中点

In [27]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids,docs)))

In [29]:
sub_docs=retriever.vectorstore.similarity_search("退货怎么处理")
sub_docs

[Document(metadata={'doc_id': '8a81550b-8430-4a84-afbc-8621ec247b51'}, page_content='华为商城的退换货服务政策主要包括以下几点：\n\n1. **总则**：华为商城严格遵循国家《三包》法规，并提供“7天无理由退货、15天质量问题换货、一年保修”的售后保障服务。\n\n2. **退换货时效**：\n   - 7天无理由退货：自收货次日起7天内，若商品完好（未激活、无受损）的情况下可申请退货。\n   - 15天质量问题换货：自收货次日起15天内，若产品出现性能故障，经华为官方售后检测属实，可免费更换同型号同规格的新机。\n   - 一年保修：自收货次日起一年内，商品出现非人为损坏的性能故障，提供免费维修服务。\n\n3. **“商品完好”判定标准**：\n   - 申请7天无理由退货时，需满足以下条件：\n     - 产品未激活：手机、平板等涉及系统激活的产品，一旦联网激活，非质量问题不支持退货。\n     - 外观完好：产品机身、配件无划痕、无磨损、无拆解记录。\n     - 包装完整：外包装、说明书、保修卡、发票及赠品需一并退回。\n\n4. **特殊产品限制**：\n   - 个性化定制产品：如激光刻字版、特殊定制外壳产品，非质量问题不支持7天无理由退货。\n   - 卫生敏感类产品：如入耳式耳机，若包装封条已拆封且无质量问题，出于卫生考虑不支持退货。\n\n5. **退款流程**：\n   - 提交申请：在“我的订单”中点击“申请售后”，上传商品实拍照片。\n   - 寄回商品：审核通过后，用户需将商品寄回指定仓库（部分地区支持顺丰上门取件）。\n   - 仓库质检：收货后1-3个工作日内完成检测。\n   - 退款到账：质检合格后，款项将按原支付路径退回，预计3-5个工作日到账。'),
 Document(metadata={'doc_id': 'cdabc3c7-380f-4480-ab07-55685934f544'}, page_content='华为商城（VMALL）的分期付款业务指南包括了业务概述、支付方式与渠道、费用与优惠说明、办理流程和常见问题等信息。用户可以通过华为商城购买华为产品时选择分期付款服务，享受灵活的还款方案。分期付款通常提供多种支付工具，如花

假设性提问

In [31]:
from langchain_core.output_parsers import JsonOutputParser

promptStr="""
'''
{doc}
'''

根据上面的文档，生成3个相关问题和回答。响应以json列表的结构返回。返回的结构参考如下：
'''
[
    {{"question":"问题1","answer":"回答1"}},
    {{"question":"问题2","answer":"回答2"}},
    {{"question":"问题3","answer":"回答3"}}
]
'''
"""

prompt=ChatPromptTemplate.from_template(promptStr)

chain=(
    {"doc": lambda x: x.page_content}
    | prompt
    | model
    | JsonOutputParser()
)

In [32]:
hypothetical_questions=chain.batch(sub_docs,{"max_concurrency":5})
hypothetical_questions

[[{'question': '华为商城的退换货服务政策中，7天无理由退货的具体条件是什么？',
   'answer': '商品未激活：手机、平板等涉及系统激活的产品，一旦联网激活，非质量问题不支持退货。外观完好：产品机身、配件无划痕、无磨损、无拆解记录。包装完整：外包装、说明书、保修卡、发票及赠品需一并退回。'},
  {'question': '华为商城的退换货服务政策中，15天质量问题换货的具体条件是什么？',
   'answer': '自收货次日起15天内，若产品出现性能故障，经华为官方售后检测属实，可免费更换同型号同规格的新机。'},
  {'question': '华为商城的退换货服务政策中，一年保修的具体条件是什么？',
   'answer': '自收货次日起一年内，商品出现非人为损坏的性能故障，提供免费维修服务。'}],
 [{'question': '分期付款的费用与优惠说明是什么？',
   'answer': '华为商城提供多种支付工具，如花呗、银行信用卡和掌上生活/其它银行 App 支付。华为定期推出‘限时免息’活动，为用户提供优惠。'},
  {'question': '办理分期付款需要提交什么订单？',
   'answer': '用户可以通过华为商城购买华为产品时选择分期付款服务，享受灵活的还款方案。分期付款通常提供多种支付工具，如花呗、银行信用卡和掌上生活/其它银行 App 支付。'},
  {'question': '分期后是否可以提前还款？',
   'answer': '分期付款通常提供多种支付工具，如花呗、银行信用卡和掌上生活/其它银行 App 支付。华为定期推出‘限时免息’活动，为用户提供优惠。'}]]

In [33]:
documents=[]
for item in hypothetical_questions:
    for obj in item:
        content="问：{}\n答：{}".format(obj['question'],obj['answer'])
        documents.append(Document(page_content=content))
documents

[Document(metadata={}, page_content='问：华为商城的退换货服务政策中，7天无理由退货的具体条件是什么？\n答：商品未激活：手机、平板等涉及系统激活的产品，一旦联网激活，非质量问题不支持退货。外观完好：产品机身、配件无划痕、无磨损、无拆解记录。包装完整：外包装、说明书、保修卡、发票及赠品需一并退回。'),
 Document(metadata={}, page_content='问：华为商城的退换货服务政策中，15天质量问题换货的具体条件是什么？\n答：自收货次日起15天内，若产品出现性能故障，经华为官方售后检测属实，可免费更换同型号同规格的新机。'),
 Document(metadata={}, page_content='问：华为商城的退换货服务政策中，一年保修的具体条件是什么？\n答：自收货次日起一年内，商品出现非人为损坏的性能故障，提供免费维修服务。'),
 Document(metadata={}, page_content='问：分期付款的费用与优惠说明是什么？\n答：华为商城提供多种支付工具，如花呗、银行信用卡和掌上生活/其它银行 App 支付。华为定期推出‘限时免息’活动，为用户提供优惠。'),
 Document(metadata={}, page_content='问：办理分期付款需要提交什么订单？\n答：用户可以通过华为商城购买华为产品时选择分期付款服务，享受灵活的还款方案。分期付款通常提供多种支付工具，如花呗、银行信用卡和掌上生活/其它银行 App 支付。'),
 Document(metadata={}, page_content='问：分期后是否可以提前还款？\n答：分期付款通常提供多种支付工具，如花呗、银行信用卡和掌上生活/其它银行 App 支付。华为定期推出‘限时免息’活动，为用户提供优惠。')]

In [34]:
vectorstore=Chroma(collection_name="Question",embedding_function=embeddings,persist_directory="./vector_store")

store=InMemoryByteStore()
id_key="doc_id"

retriever=MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids=[str(uuid.uuid4()) for _ in docs]

In [35]:
retriever.vectorstore.add_documents(documents)

['261032ff-cb2e-454d-8107-a29df6a7428a',
 '21bf0d95-38e1-4cd1-9a7f-7452d8146e02',
 'f952fb1e-712f-42b3-be98-62664b72cc04',
 '03fa9f76-5b08-4d28-8b12-30294be2556e',
 '67dc4498-fc2b-45e1-ae64-85e5e0aad5cd',
 '80da7007-435c-4577-abdd-790f75d76134']

In [36]:
retriever.vectorstore.similarity_search("退货")[0]

Document(metadata={}, page_content='问：华为商城的退换货服务政策中，7天无理由退货的具体条件是什么？\n答：商品未激活：手机、平板等涉及系统激活的产品，一旦联网激活，非质量问题不支持退货。外观完好：产品机身、配件无划痕、无磨损、无拆解记录。包装完整：外包装、说明书、保修卡、发票及赠品需一并退回。')